# 01. 설정 및 연결

이 노트북에서 다루는 내용:
- 데이터베이스 연결 확인
- 서비스 가용성 확인 (Ollama, Langfuse)
- 데이터베이스 스키마(Schema) 탐색
- 샘플 데이터 확인

## 1. 라이브러리 임포트

In [ ]:
import sys
sys.path.append('/workspace')

import os
import requests
from src.utils.db_utils import DatabaseConnection, get_database_context
import pandas as pd

print("✓ Libraries imported successfully")

## 2. 환경 변수 확인

In [ ]:
# 환경 변수 확인
env_vars = {
    'POSTGRES_HOST': os.getenv('POSTGRES_HOST'),
    'POSTGRES_PORT': os.getenv('POSTGRES_PORT'),
    'POSTGRES_DB': os.getenv('POSTGRES_DB'),
    'OLLAMA_HOST': os.getenv('OLLAMA_HOST'),
}

print("Environment Variables:")
for key, value in env_vars.items():
    print(f"  {key}: {value}")

## 3. 데이터베이스 연결 테스트

In [ ]:
# 데이터베이스 초기화 connection
db = DatabaseConnection()

# 연결 테스트
try:
    conn = db.get_connection()
    conn.close()
    print("✓ Database connection successful!")
except Exception as e:
    print(f"✗ Database connection failed: {e}")

## 4. 데이터베이스 스키마 탐색

In [ ]:
# 모든 테이블 가져오기
tables = db.get_all_tables()
print(f"Found {len(tables)} tables:")
for table in tables:
    print(f"  - {table}")

In [ ]:
# 각 테이블 스키마 검사
for table in tables:
    print(f"\n{'='*60}")
    print(f"Table: {table}")
    print(f"{'='*60}")
    schema = db.get_table_schema(table)
    display(schema)
    
    # 행 개수 표시
    count = db.execute_query(f"SELECT COUNT(*) as count FROM {table}")[0]['count']
    print(f"Row count: {count}")

## 5. 샘플 데이터 보기

In [ ]:
# 주요 테이블의 샘플 데이터 보기
main_tables = ['departments', 'employees', 'customers', 'projects', 'sales']

for table in main_tables:
    print(f"\n{'='*60}")
    print(f"Sample data from: {table}")
    print(f"{'='*60}")
    df = db.get_table_sample(table, limit=3)
    display(df)

## 6. Ollama 서비스 확인

In [ ]:
# Ollama 실행 중인지 확인
ollama_host = os.getenv('OLLAMA_HOST', 'http://localhost:11434')

try:
    response = requests.get(f"{ollama_host}/api/tags", timeout=5)
    if response.status_code == 200:
        models = response.json().get('models', [])
        print(f"✓ Ollama is running at {ollama_host}")
        print(f"Available models: {len(models)}")
        for model in models:
            print(f"  - {model.get('name')}")
    else:
        print(f"✗ Ollama responded with status {response.status_code}")
except Exception as e:
    print(f"✗ Cannot connect to Ollama: {e}")
    print("To install a model, run in terminal: docker exec -it text2sql-ollama ollama pull llama2")

## 7. LLM을 위한 데이터베이스 컨텍스트 가져오기

In [ ]:
# Text2SQL에 사용될 완전한 데이터베이스 컨텍스트 가져오기
context = get_database_context()
print(context)

## 8. pgvector 확장 기능 확인

In [ ]:
# pgvector 확장 기능이 설치되어 있는지 확인
query = "SELECT * FROM pg_extension WHERE extname = 'vector'"
result = db.execute_query(query)

if result:
    print("✓ pgvector extension is installed")
    print(f"Version: {result[0].get('extversion', 'unknown')}")
else:
    print("✗ pgvector extension not found")

## 요약

다음 항목들을 확인했습니다:
- ✓ 데이터베이스 연결
- ✓ 데이터베이스 스키마 및 샘플 데이터
- ✓ Ollama 서비스 가용성
- ✓ pgvector 확장 기능

다음 단계: `02_embedding_and_rag.ipynb`로 이동하여 임베딩(Embedding)과 RAG를 실습합니다.